# 背景

在东财app中，选择近一个月机构调研次数3次以上；股吧人气排名前1000名；换手率>5%；10日涨跌幅0%~10%的股票进行跟踪，反复炒作。

## 买入

30分钟线均处于低位，并且30分钟得到一周期确认。注意如果日线处于明显的下降通道、特别是缓降通道时，还要参考均线压制信号。比如东土科技在11月2日14：30点给出RSI低位确认信号，但由于上方多均线压制，日线看RSI并不超跌（30.2, 成为最低点的概率为16%)，所以还要再等，到2021年11月4日尾盘更合适。

<img src="https://images.jieyu.ai/images/202111/20211104215554.png" width="350"/>

完全符合上述pattern的有：金杯电工（11月3日）、运达股份（11月3日）、芯能科技（11月3日）、华明装备（11月3日）、东方电热（11月3日）、合兴股份（10月29日）等。

英科医疗（11月2日）RSI处于低位，次日涨14%。但下一周期上涨幅度小于1%，不知道能否算确认。类似RSI30处于低位后，第二天30分钟类拉涨停的还有一些。


## 卖出

日线处于高位并且30分钟也处于高位

## 淘汰

股票入池后30日移除

In [2]:
from alpha.notebook import *
import traceback
import json

from alpha.core.rsi_stats import RsiStats, rsiday, rsi30
await init_notebook()

events = {
    tf.date2int(arrow.now()): {}
}

In [3]:
def load_fired_events(path="/apps/alpha/data/机构调研-信号.txt"):
    # 当天和头一天已报过的信号不重复报
    today = tf.date2int(arrow.now())

    with open(path, "r") as f:
        events = json.load(f)
        
    today = tf.date2int(arrow.now())
    if today not in events:
        events[today] = {}
        
    return events

def save_fire_events(path="/apps/alpha/data/机构调研-信号.txt"):
    global events
    with open(path, "w") as f:
        json.dump(events, f)
    
def is_fired(name):
    global events
    
    end = arrow.now().date()
    
    start = tf.date2int(tf.day_shift(end, -1))
    end = tf.date2int(end)
    
    if name in events[end]:
        return True
    
    if start in events and name in events[start]:
        return True
        
    return False

def remember(name, params: List):
    end = tf.date2int(arrow.now())
    events[end][name] = params
    
def print_list():
    df = load_pool()
    groups = df.groupby('date').groups
    print("最新监控名单:\n")
    for tm in groups:
        index = groups[tm].tolist()
        print(tm, " ".join(df.loc[index]["name"].tolist()))


In [3]:
def load_pool(filter_aged=True):
    if os.path.exists("/apps"):
        path =  "/apps/alpha/data/机构调研.txt"
    else:
        path = "/Users/aaronyang/workspace/alpha/data/机构调研.txt"

    with open(path, "r") as f:
        content = f.readlines()
        
    stocks = {}
    for line in content:
        line = line.strip()
        if line.startswith("#") or len(line) == 0:
            continue
        
        tm, *names = line.split(" ")
        for name in names:
            if name in stocks:
                continue
                
            stocks[name] = arrow.get(tm).date()
            
    df = pd.DataFrame().from_dict(stocks.items())
    df.columns = ["name", "date"]
    
    if filter_aged:
        time_mark = arrow.now().shift(days=-30).date()
        df = df[df.date > time_mark]
        
    return df

def hot():
    if os.path.exists("/apps"):
        path =  "/apps/alpha/data/机构调研.txt"
    else:
        path = "/Users/aaronyang/workspace/alpha/data/机构调研.txt"

    with open(path, "r") as f:
        content = f.readlines()
        
    result = {}
    aged = arrow.now().shift(days=-30)
    for line in content:
        line = line.strip()
        if line.startswith("#") or len(line) == 0:
            continue
        
        tm, *names = line.split(" ")
        if arrow.get(tm) < aged:
            continue
        for name in names:
            count = result.get(name, 0)
            result[name] = count + 1
        
    arr = [(k,v) for k,v in result.items()]
    return sorted(arr, key=lambda x:x[1])

In [4]:
def trigger(code, name, bars, results, frame_type):
    """30分钟RSI低点确认，或者由跌转升
    
    低点确认原则：
    1. RSI低位(prsi < 10%)且有长下影
    2. 前5周期内有RSI低位且自低位后，有1%以上涨幅
    3. RSI不在低位，但系30日以内最低点
    4. 其它可以确认低位的方法
    """
    if is_fired(name): #一个信号两天只报一次
        return
    
    close = bars["close"]
    
    rsi = relative_strength_index(close, 6)
    min_rsi = np.min(rsi[-5:])
    ipos = np.argmin(rsi[-5:])
    
    rf = reversing(close)
        
    # 低位rsi
    pmin_rsi = rsi30.get_proba(code, min_rsi)

    low_rsi = (pmin_rsi is not None and (pmin_rsi < 0.1 or abs(min_rsi - np.min(rsi) < 1e-4)))
    
    # 出现低位rsi后，上涨 > 1%,但不追涨（后面跌下来可以继续报）
    low = bars["low"][-5:]
    adv = round((close[-1] / low[ipos - 5] - 1),3)
    is_up = (0.03 >= adv >= 0.01)

    frame = str(bars["frame"][-1])[5:-9]
    #print(name, frame, pmin_rsi, min_rsi, adv)

    if low_rsi and is_up or rf == 1:
        remember(name, [name, code, frame, pmin_rsi, min_rsi, adv, rf])
        results.append([name, code, frame, pmin_rsi, min_rsi, adv, rf])

In [5]:
async def strategy(tm=None):
    records = load_pool()
    names = records["name"].tolist()
    codes = [name_to_code(name) for name in names]
    #codes = ["002270.XSHE"]
    r2 = await scan(trigger, 80, '30m', tm = tm, codes=codes)
    if len(r2):
        columns = ["name", "code", "frame", "PRSI", "RSI", "adv", "d2"]
        df = pd.DataFrame(r2, columns=columns)
        display(df)
        await nb_say("机构调研股有新的信号。")

# 每日监控

In [4]:
load_fired_events()
print_list()
await scheduler(strategy)
print("结束一天的机构调研股监控。")

NameError: name 'load_fired_events' is not defined

# 入池后30天内表现评估

In [37]:
async def evaluate(end=None):    
    results = []
    end = arrow.get(end).date() if end else arrow.now().date()
    for name, start in load_pool(filter_aged=False).to_numpy():
        try:
            code = name_to_code(name)
            n = tf.count_day_frames(start, end)
            if n < 3:
                continue
                
            bars = await get_bars(code, n, '1d', end)

            buy = bars[1]["open"]
            close = bars["close"][2:]

            pmax = max(close)/buy - 1
            pmin = min(close)/buy - 1

            row = [name, code, start, pmax, pmin]
            for i in [1, 3, 5, 8, 13, 18]:
                if i > len(close) - 1:
                    row.append(np.nan)
                else:
                    row.append(close[i]/buy - 1)
            results.append(row)
        except Exception:
            pass
        
    columns = ["name", "code", "date", "max_gains", "max_loss", "1", "3", "5", "8", "13", "18"]
    return pd.DataFrame(results, columns=columns)

pd.options.display.max_rows = 120
df = await evaluate()

In [60]:
df[(~df["5"].isna())&(df.date>datetime.date(2021,11,6))]

,name,code,date,max_gains,max_loss,1,3,5,8,13,18
83,国机精工,002046.XSHE,2021-11-10,0.014525,-0.097765,-0.040223,-0.025140,-0.026257,-0.039106,NaN,NaN
84,高澜股份,300499.XSHE,2021-11-10,0.144701,-0.010759,0.031737,0.032813,0.105971,0.112426,NaN,NaN
85,水晶光电,002273.XSHE,2021-11-10,0.163064,0.042001,0.068561,0.064855,0.114886,0.130945,NaN,NaN
86,金风科技,002202.XSHE,2021-11-10,0.007903,-0.056902,-0.032139,-0.021602,0.001581,-0.056902,NaN,NaN
87,天能重工,300569.XSHE,2021-11-10,0.296603,-0.009113,-0.009113,0.138360,0.296603,0.196355,NaN,NaN
88,新劲刚,300629.XSHE,2021-11-12,0.032464,-0.060073,-0.021541,-0.060073,-0.058252,0.032464,NaN,NaN
89,四方达,300179.XSHE,2021-11-12,0.105833,0.019167,0.048333,0.056667,0.098333,0.019167,NaN,NaN
90,隆利科技,300752.XSHE,2021-11-12,0.076981,-0.039805,0.021404,-0.039805,-0.012017,-0.021405,NaN,NaN


In [33]:
pd.options.display.max_rows = 200
df.sort_values("date",ascending=False)

,name,code,date,max_gains,max_loss,3,5,8,10,18,20
124,康达新材,002669.XSHE,2021-11-23,-0.027557,-0.027557,NaN,NaN,NaN,NaN,NaN,NaN
123,华邦健康,002004.XSHE,2021-11-23,-0.003968,-0.003968,NaN,NaN,NaN,NaN,NaN,NaN
122,中科三环,000970.XSHE,2021-11-23,0.052668,0.052668,NaN,NaN,NaN,NaN,NaN,NaN
121,泰和新材,002254.XSHE,2021-11-23,0.015507,0.015507,NaN,NaN,NaN,NaN,NaN,NaN
120,爱施德,002416.XSHE,2021-11-23,-0.030781,-0.030781,NaN,NaN,NaN,NaN,NaN,NaN
115,龙磁科技,300835.XSHE,2021-11-22,0.073684,-0.006579,NaN,NaN,NaN,NaN,NaN,NaN
110,天地在线,002995.XSHE,2021-11-22,-0.014902,-0.023145,NaN,NaN,NaN,NaN,NaN,NaN
112,亚太科技,002540.XSHE,2021-11-22,-0.061881,-0.066832,NaN,NaN,NaN,NaN,NaN,NaN
113,鸿利智汇,300219.XSHE,2021-11-22,0.053750,0.046875,NaN,NaN,NaN,NaN,NaN,NaN
114,英诺激光,301021.XSHE,2021-11-22,0.001302,-0.002386,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df = load_pool()
df[df.date>arrow.get('2021-11-23').date()]

,name,date
127,宸展光电,2021-11-24
128,健麾信息,2021-11-24
129,张小泉,2021-11-24
130,上海艾录,2021-11-24
131,信维通信,2021-11-24
132,欣贺股份,2021-11-24
133,吉电股份,2021-11-24
134,微光股份,2021-11-24
135,中来股份,2021-11-24
136,富瀚微,2021-11-25


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/cfg4py/core.py", line 357, in _load_from_local_file
    ext = _guess_extension()
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/cfg4py/core.py", line 305, in _guess_extension
    raise FileNotFoundError(msg)
FileNotFoundError: No config files present, or file format is not yaml.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/watchdog/observers/api.py", line 199, in run
    self.dispatch_events(self.event_queue, self.timeout)
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/watchdog/observers/api.py", line 372, in dispatch_events
    handler.dispa

In [5]:
load_pool()

,name,date
51,中闽能源,2021-11-04
52,佳禾食品,2021-11-04
53,智光电气,2021-11-04
54,申昊科技,2021-11-04
55,联合光电,2021-11-04
...,...,...
164,海油发展,2021-12-01
165,劲嘉股价,2021-12-01
166,川金诺,2021-12-01
167,永兴材料,2021-12-01
